In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/data_labeling.csv')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# 시드 고정
import random
import torch
import numpy as np

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
df_title = df[['doc_title', 'label']]
df_content = df[['doc_contents', 'label']]

## koelectra

### analyze contents

####epoch test

In [ ]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model_koelectra = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkp

In [ ]:
from datasets import Dataset
# 'doc_content' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    examples['doc_contents'] = [text + ' </s>' for text in examples['doc_contents']]
    return tokenizer(examples['doc_contents'], padding='max_length', max_length=512, truncation=True)

# DataFrame을 Hugging Face Dataset으로 변환
dataset_title = Dataset.from_pandas(df_content)

# 데이터셋에 토크나이저 적용
tokenized_dataset = dataset_title.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/39987 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset
# 'doc_content' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    examples['doc_contents'] = [text + ' </s>' for text in examples['doc_contents']]
    return tokenizer(examples['doc_contents'], padding='max_length', max_length=512, truncation=True)

# DataFrame을 Hugging Face Dataset으로 변환
dataset_title = Dataset.from_pandas(df_content)

# 데이터셋에 토크나이저 적용
tokenized_dataset = dataset_title.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/39987 [00:00<?, ? examples/s]

In [ ]:
# loss 저장
from transformers import TrainerCallback

class SaveLossCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if "loss" in logs:
            self.train_losses.append(logs["loss"])
        if "eval_loss" in logs:
            self.eval_losses.append(logs["eval_loss"])


In [ ]:
from transformers import Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

for param in model_koelectra.parameters():
    param.data = param.data.contiguous()

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_koelectra,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
train_loss_koelectra = save_loss_callback.train_losses
eval_loss_koelectra = save_loss_callback.eval_losses
lossdata_electra = pd.DataFrame({'epoch':[i+1 for i in range(10)],'train_loss_koelectra': train_loss_koelectra, 'eval_loss_koelectra': eval_loss_koelectra})
lossdata_electra.to_csv('lossdata_contents_electra.csv')

####epoch 2로 모델 재학습

In [ ]:
from transformers import Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

model_koelectra_context = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

for param in model_koelectra_context.parameters():
    param.data = param.data.contiguous()

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_koelectra_context,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.201000,0.157170
2,0.147900,0.161686


TrainOutput(global_step=2000, training_loss=0.17443810272216798, metrics={'train_runtime': 1472.581, 'train_samples_per_second': 43.446, 'train_steps_per_second': 1.358, 'total_flos': 1.683331909982208e+16, 'train_loss': 0.17443810272216798, 'epoch': 2.0})

### analyze title

####epoch test

In [ ]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
model_koelectra = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from datasets import Dataset
# 'doc_title' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    examples['doc_title'] = [text + ' </s>' for text in examples['doc_title']]
    return tokenizer(examples['doc_title'], padding='max_length', max_length=256, truncation=True)

# DataFrame을 Hugging Face Dataset으로 변환
dataset_title = Dataset.from_pandas(df_title)

# 데이터셋에 토크나이저 적용
tokenized_dataset = dataset_title.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

Map:   0%|          | 0/39987 [00:00<?, ? examples/s]

In [ ]:
# loss 저장
from transformers import TrainerCallback

class SaveLossCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if "loss" in logs:
            self.train_losses.append(logs["loss"])
        if "eval_loss" in logs:
            self.eval_losses.append(logs["eval_loss"])


In [ ]:
from transformers import Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

for param in model_koelectra.parameters():
    param.data = param.data.contiguous()

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_koelectra,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.488800,0.445866
2,0.407900,0.447759
3,0.359000,0.468784
4,0.315300,0.546391
5,0.288200,0.674143
6,0.261300,0.724622
7,0.238800,0.798933


Epoch,Training Loss,Validation Loss
1,0.488800,0.445866
2,0.407900,0.447759
3,0.359000,0.468784
4,0.315300,0.546391
5,0.288200,0.674143
6,0.261300,0.724622
7,0.238800,0.798933
8,0.219600,0.872394
9,0.207100,0.998000
10,0.192300,1.078408


TrainOutput(global_step=10000, training_loss=0.29781556396484377, metrics={'train_runtime': 14782.2144, 'train_samples_per_second': 21.64, 'train_steps_per_second': 0.676, 'total_flos': 4.20832977495552e+16, 'train_loss': 0.29781556396484377, 'epoch': 10.0})

In [ ]:
train_loss_koelectra = save_loss_callback.train_losses
eval_loss_koelectra = save_loss_callback.eval_losses
lossdata_electra = pd.DataFrame({'epoch':[i+1 for i in range(10)],'train_loss_koelectra': train_loss_koelectra, 'eval_loss_koelectra': eval_loss_koelectra})
lossdata_electra.to_csv('lossdata_electra.csv')

####epoch 2 재학습

In [ ]:
from transformers import Trainer, TrainingArguments

model_koelectra_title = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-base-v3-discriminator", num_labels=2)

save_loss_callback = SaveLossCallback()

for param in model_koelectra_title.parameters():
    param.data = param.data.contiguous()

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_koelectra_title,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.491500,0.440835
2,0.400600,0.444670


TrainOutput(global_step=2000, training_loss=0.4460820465087891, metrics={'train_runtime': 700.4074, 'train_samples_per_second': 91.344, 'train_steps_per_second': 2.855, 'total_flos': 8416659549911040.0, 'train_loss': 0.4460820465087891, 'epoch': 2.0})

## kobart

### analyze contents

In [ ]:
from transformers import PreTrainedTokenizerFast
from datasets import Dataset

# KoBART 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 'doc_contents' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    tokens = tokenizer(examples['doc_contents'], padding='max_length', max_length=512, truncation=True)

    eos_token_id = tokenizer.eos_token_id
    for i in range(len(tokens['input_ids'])):
        # 길이가 max_length 이하인 경우에는 eos 토큰 추가
        if len(tokens['input_ids'][i]) < 512:
            tokens['input_ids'][i][-1] = eos_token_id  # 마지막에 eos 토큰 추가
        # 만약 길이가 정확히 512인 경우, 마지막에 eos 토큰이 없을 수도 있으므로 마지막 토큰을 교체
        elif tokens['input_ids'][i][-1] != eos_token_id:
            tokens['input_ids'][i][-1] = eos_token_id  # 마지막 토큰을 eos로 교체

    return tokens

# DataFrame을 Hugging Face Dataset으로 변환
hf_dataset = Dataset.from_pandas(df_content)

# 데이터셋에 토크나이저 적용
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# 모델에 필요한 형식으로 설정 (input_ids, attention_mask, labels)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# train/test split
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/39987 [00:00<?, ? examples/s]

In [ ]:
# loss 저장
from transformers import TrainerCallback

class SaveLossCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if "loss" in logs:
            self.train_losses.append(logs["loss"])
        if "eval_loss" in logs:
            self.eval_losses.append(logs["eval_loss"])


In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

# KoBART 모델 불러오기
model_kobart = BartForSequenceClassification.from_pretrained('gogamza/kobart-base-v2', num_labels=2)

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_kobart,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/495M [00:00<?, ?B/s]

Some weights of BartForSequenceClassification were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.023200,0.011393
2,0.007100,0.009395
3,0.005600,0.016522
4,0.003900,0.008165
5,0.002300,0.011037
6,0.001500,0.011565
7,0.001100,0.013971
8,0.001000,0.015445
9,0.000500,0.013029
10,0.000400,0.013668


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=10000, training_loss=0.00464970295727253, metrics={'train_runtime': 7769.9091, 'train_samples_per_second': 41.17, 'train_steps_per_second': 1.287, 'total_flos': 9.81061851958272e+16, 'train_loss': 0.00464970295727253, 'epoch': 10.0})

In [ ]:
lossdata_kobart = pd.DataFrame({'epoch':[i+1 for i in range(10)],'train_loss_kobart':save_loss_callback.train_losses, 'eval_loss_kobart': save_loss_callback.eval_losses})
lossdata_kobart.to_csv('lossdata_contents_kobart.csv')

####epoch2 재학습

In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

# KoBART 모델 불러오기
model_kobart_contents = BartForSequenceClassification.from_pretrained('gogamza/kobart-base-v2', num_labels=2)

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_kobart_contents,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.019900,0.010602
2,0.006400,0.008429


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=2000, training_loss=0.013143981456756592, metrics={'train_runtime': 1556.0018, 'train_samples_per_second': 41.117, 'train_steps_per_second': 1.285, 'total_flos': 1.962123703916544e+16, 'train_loss': 0.013143981456756592, 'epoch': 2.0})

###analyze title

In [ ]:
from transformers import PreTrainedTokenizerFast
from datasets import Dataset

# KoBART 토크나이저 불러오기
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 'doc_title' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    examples['doc_title'] = [text + ' </s>' for text in examples['doc_title']]
    return tokenizer(examples['doc_title'], padding='max_length', max_length=128, truncation=True)

# DataFrame을 Hugging Face Dataset으로 변환
hf_dataset = Dataset.from_pandas(df_title)

# 데이터셋에 토크나이저 적용
tokenized_dataset = hf_dataset.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
split_dataset = tokenized_dataset.train_test_split(test_size=0.2)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/39987 [00:00<?, ? examples/s]

In [ ]:
# loss 저장
from transformers import TrainerCallback

class SaveLossCallback(TrainerCallback):
    def __init__(self):
        self.train_losses = []
        self.eval_losses = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if "loss" in logs:
            self.train_losses.append(logs["loss"])
        if "eval_loss" in logs:
            self.eval_losses.append(logs["eval_loss"])


In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

# KoBART 모델 불러오기
model_kobart = BartForSequenceClassification.from_pretrained('gogamza/kobart-base-v2', num_labels=2)

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_kobart,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

In [ ]:
lossdata_kobart = pd.DataFrame({'epoch':[i+1 for i in range(10)],'train_loss_kobart':save_loss_callback.train_losses, 'eval_loss_kobart': save_loss_callback.eval_losses})
lossdata_kobart.to_csv('lossdata_kobart.csv')

####epoch2 재학습

In [ ]:
from transformers import BartForSequenceClassification, Trainer, TrainingArguments

save_loss_callback = SaveLossCallback()

# KoBART 모델 불러오기
model_kobart_title = BartForSequenceClassification.from_pretrained('gogamza/kobart-base-v2', num_labels=2)

# 학습 인자 정의
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir='./logs',  # 로그 저장 경로
)

# Trainer 초기화
trainer = Trainer(
    model=model_kobart_title,
    args=training_args,
    train_dataset=split_dataset['train'],
    eval_dataset=split_dataset['test'],
    callbacks=[save_loss_callback]  # 콜백 추가
)

# 모델 파인튜닝 실행
trainer.train()

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at gogamza/kobart-base-v2 and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.514600,0.477827
2,0.426300,0.463950


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

TrainOutput(global_step=2000, training_loss=0.47044963073730467, metrics={'train_runtime': 417.9592, 'train_samples_per_second': 153.072, 'train_steps_per_second': 4.785, 'total_flos': 4905309259791360.0, 'train_loss': 0.47044963073730467, 'epoch': 2.0})

## 평가

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/data/test_labeling.csv')
test

,doc_title,doc_contents,doc_class,label
0,강원도공예협동조합 박경수 이사장 선임,강원도공예협동조합 신임 이사장에 박경수(65·사진) 경춘필방 대표가 지난 12일 선...,경제,0
1,내달 방미 앞둔 文 “바이든 대통령 빨리 북과 대화를”,문재인 대통령이 다음 달 하순 미국 워싱턴에서 첫 한·미 정상회담을 하게 될 조 바...,정치,0
2,성민♥김사은 TMI 고백… #만남 #2세계획 #미스트롯,그룹 슈퍼주니어 출신 성민과 배우 김사은 부부가 결혼 7년차 일상을 공개했다. \n...,사회,0
3,"연산1동 밥정봉사단, '할매랑 아이랑 우리는 밥 짝꿍' 운영",연제구 연산1동(동장 박순미)은 지난 24일 홀로 어르신과 양육공백 우려 가정 아이...,사회,0
4,이태원 클럽 방문 숨기고 학원 강의… 인천서 집단감염 발생,인천에서 서울 이태원 클럽발 신종 코로나바이러스 감염증(코로나19) 집단감염이 발생...,사회,0
...,...,...,...,...
15938,"[헬스 프리즘] 할머니 맛집, 며느리가 물려받고 손님 줄어든 까닭은?",코로나19가 일부 감염 환자에게 미각과 후각을 상실하게 한다는 점이 주목을 받았다....,사회,1
15939,미국 시장에서 500만 달러 수출 계약 추진,경기도는 코로나19로 인한 수출 파고를 넘기 위해 올해 첫 미국 온라인 통상촉진단을...,경제,1
15940,"문 대통령, P4G 서울정상회의 참석 예정",문재인 대통령은 이번 주 30일부터 31일까지 서울에서 열리는 '2021 P4G 서...,정치,1
15941,"성남시, 착한 임대인 재산세 감면 연장 및 업종·규모 제한 없애","성남시는 올해까지 '착한 임대인'에 대한 재산세 감면 조치를 연장하고, 임대료 인하...",사회,1


In [ ]:
test_content = test[['doc_contents','label']]
test_title = test[['doc_title','label']]

In [97]:
import torch
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

# 정확도 계산 함수
def calculate_accuracy(predictions, labels):
    return accuracy_score(labels, predictions)

# AUROC 계산 함수
def calculate_auroc(prob_predictions, labels):
    return roc_auc_score(labels, prob_predictions)

# 정밀도(Precision) 계산 함수
def calculate_precision(predictions, labels):
    return precision_score(labels, predictions)

# 재현율(Recall) 계산 함수
def calculate_recall(predictions, labels):
    return recall_score(labels, predictions)

# F1 Score 계산 함수
def calculate_f1(predictions, labels):
    return f1_score(labels, predictions)

####contents 평가

In [ ]:
# electra_contents tokenizer
from datasets import Dataset

tokenizer_KoELECTRA = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# 'doc_content' 컬럼을 토큰화하는 함수 정의
def tokenize_function_contents(examples):
    examples['doc_contents'] = [text + ' </s>' for text in examples['doc_contents']]
    return tokenizer_KoELECTRA(examples['doc_contents'], padding='max_length', max_length=512, truncation=True)

# DataFrame을 Hugging Face Dataset으로 변환
dataset_content = Dataset.from_pandas(test_content)

# 데이터셋에 토크나이저 적용
tokenized_test_contents_electra = dataset_content.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_test_contents_electra = tokenized_test_contents_electra.rename_column("label", "labels")
tokenized_test_contents_electra.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

In [ ]:
# bart content tokenizer
from transformers import PreTrainedTokenizerFast
from datasets import Dataset

# KoBART 토크나이저 불러오기
tokenizer_KoBART = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

# 'doc_contents' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    tokens = tokenizer_KoBART(examples['doc_contents'], padding='max_length', max_length=512, truncation=True)

    eos_token_id = tokenizer.eos_token_id
    for i in range(len(tokens['input_ids'])):
        # 길이가 max_length 이하인 경우에는 eos 토큰 추가
        if len(tokens['input_ids'][i]) < 512:
            tokens['input_ids'][i][-1] = eos_token_id  # 마지막에 eos 토큰 추가
        # 만약 길이가 정확히 512인 경우, 마지막에 eos 토큰이 없을 수도 있으므로 마지막 토큰을 교체
        elif tokens['input_ids'][i][-1] != eos_token_id:
            tokens['input_ids'][i][-1] = eos_token_id  # 마지막 토큰을 eos로 교체

    return tokens

# DataFrame을 Hugging Face Dataset으로 변환
dataset_content = Dataset.from_pandas(test_content)

# 데이터셋에 토크나이저 적용
tokenized_test_contents_kobart = dataset_content.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_test_contents_kobart = tokenized_test_contents_kobart.rename_column("label", "labels")

# 모델에 필요한 형식으로 설정 (input_ids, attention_mask, labels)
tokenized_test_contents_kobart.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def move_to_device(example):
    return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in example.items()}

# 각 텐서를 GPU로 이동
tokenized_test_contents_kobart = tokenized_test_contents_kobart.map(move_to_device)
tokenized_test_contents_electra = tokenized_test_contents_electra.map(move_to_device)

# 모델을 GPU로 이동
model_kobart_contents = model_kobart_contents.to(device)
model_koelectra_context = model_koelectra_context.to(device)

Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

In [101]:
def ensemble_predict_prob_and_label(kobart_inputs, koelectra_inputs, kobart_model, koelectra_model):
    # KoBART 모델 예측 (확률 출력)
    kobart_outputs = kobart_model(
        input_ids=kobart_inputs['input_ids'],
        attention_mask=kobart_inputs['attention_mask']
    )
    kobart_probs = torch.softmax(kobart_outputs.logits, dim=-1)

    # KoELECTRA 모델 예측 (확률 출력)
    koelectra_outputs = koelectra_model(
        input_ids=koelectra_inputs['input_ids'],
        attention_mask=koelectra_inputs['attention_mask']
    )
    koelectra_probs = torch.softmax(koelectra_outputs.logits, dim=-1)

    # Soft voting (평균 확률 계산)
    ensemble_probs = (kobart_probs + koelectra_probs) / 2

    # 양성 클래스에 대한 확률 반환 (보통 index 1에 해당)
    positive_class_probs = ensemble_probs[:, 1]

    # 클래스 레이블 반환 (가장 높은 확률의 클래스를 선택)
    class_preds = torch.argmax(ensemble_probs, dim=-1)

    return positive_class_probs, class_preds

In [102]:
# 앙상블 모델 soft voting 기반
def get_prob_and_label_predictions(tokenized_kobart, tokenized_electra, model_kobart, model_electra):
    prob_preds = []
    class_preds = []
    true_labels = []

    # tqdm을 사용해 데이터셋 반복 시 진행 상황 표시
    for i in tqdm(range(len(tokenized_kobart)), desc="Processing Predictions"):
        # KoBART 데이터 추출
        kobart_input = {
            'input_ids': tokenized_kobart[i]['input_ids'].unsqueeze(0).to(device),
            'attention_mask': tokenized_kobart[i]['attention_mask'].unsqueeze(0).to(device)
        }

        # KoELECTRA 데이터 추출
        electra_input = {
            'input_ids': tokenized_electra[i]['input_ids'].unsqueeze(0).to(device),
            'attention_mask': tokenized_electra[i]['attention_mask'].unsqueeze(0).to(device)
        }

        # 모델 확률 및 클래스 레이블 예측
        prob_prediction, class_prediction = ensemble_predict_prob_and_label(kobart_input, electra_input, model_kobart, model_electra)

        # 실제 라벨 저장
        true_labels.append(tokenized_kobart[i]['labels'].item())

        # 확률 예측 및 클래스 레이블 저장
        prob_preds.append(prob_prediction.item())
        class_preds.append(class_prediction.item())

    return prob_preds, class_preds, true_labels

In [103]:
# 예측 수행
prob_predictions, class_predictions, true_labels = get_prob_and_label_predictions(
    tokenized_test_contents_kobart, tokenized_test_contents_electra, model_kobart_contents, model_koelectra_context
)

accuracy = calculate_accuracy(class_predictions, true_labels)
print(f"Ensemble Model Accuracy: {accuracy:.4f}")

auroc = calculate_auroc(prob_predictions, true_labels)
print(f"Ensemble Model AUROC: {auroc:.4f}")

precision = calculate_precision(class_predictions, true_labels)
print(f"Ensemble Model Precision: {precision:.4f}")

recall = calculate_recall(class_predictions, true_labels)
print(f"Ensemble Model Recall: {recall:.4f}")

f1 = calculate_f1(class_predictions, true_labels)
print(f"Ensemble Modell F1 Score: {f1:.4f}")

Processing Predictions: 100%|██████████| 15943/15943 [07:39<00:00, 34.70it/s]


Ensemble Model Accuracy: 0.9992
Ensemble Model AUROC: 0.9996
Ensemble Model Precision: 0.9990
Ensemble Model Recall: 0.9995
Ensemble Modell F1 Score: 0.9992


In [105]:
input_ids = tokenized_test_contents_electra['input_ids'].to(device)
attention_mask = tokenized_test_contents_electra['attention_mask'].to(device)
true_labels = tokenized_test_contents_electra['labels'].to(device)

batch_size = 16

# 예측 결과 저장용 리스트
all_class_preds = []
all_prob_preds = []
all_true_labels = []

# 데이터 배치 처리
model_koelectra_context.eval()  # 평가 모드
with torch.no_grad():
    for i in tqdm(range(0, len(input_ids), batch_size)):
        # 배치 추출
        input_ids_batch = input_ids[i:i+batch_size].to(device)
        attention_mask_batch = attention_mask[i:i+batch_size].to(device)
        true_labels_batch = true_labels[i:i+batch_size].to(device)

        # 모델 예측
        outputs = model_koelectra_context(input_ids=input_ids_batch, attention_mask=attention_mask_batch)

        # 각 클래스에 대한 확률 계산 (Softmax)
        probs = torch.softmax(outputs.logits, dim=-1)

        # 클래스 레이블 예측 (가장 높은 확률의 클래스를 선택)
        class_preds = torch.argmax(probs, dim=-1)

        # 양성 클래스 확률 (index 1)
        positive_class_probs = probs[:, 1]

        # 결과 저장
        all_class_preds.extend(class_preds.cpu().numpy())
        all_prob_preds.extend(positive_class_probs.cpu().numpy())
        all_true_labels.extend(true_labels_batch.cpu().numpy())

accuracy = calculate_accuracy(all_class_preds, all_true_labels)
print(f"KoELECTRA Model Accuracy: {accuracy:.4f}")

auroc = calculate_auroc(all_prob_preds, all_true_labels)
print(f"KoELECTRA Model AUROC: {auroc:.4f}")

precision = calculate_precision(all_class_preds, all_true_labels)
print(f"KoELECTRA Model Precision: {precision:.4f}")

recall = calculate_recall(all_class_preds, all_true_labels)
print(f"KoELECTRA Model Recall: {recall:.4f}")

f1 = calculate_f1(all_class_preds, all_true_labels)
print(f"KoELECTRA Model F1 Score: {f1:.4f}")

100%|██████████| 997/997 [01:54<00:00,  8.72it/s]


KoELECTRA Model Accuracy: 0.8029
KoELECTRA Model AUROC: 0.8845
KoELECTRA Model Precision: 0.7649
KoELECTRA Model Recall: 0.8729
KoELECTRA Model F1 Score: 0.8153


In [106]:
input_ids = tokenized_test_contents_kobart['input_ids'].to(device)
attention_mask = tokenized_test_contents_kobart['attention_mask'].to(device)
true_labels = tokenized_test_contents_kobart['labels'].to(device)

batch_size = 16

# 예측 결과 저장용 리스트
all_class_preds = []
all_prob_preds = []
all_true_labels = []

# 데이터 배치 처리
model_kobart_contents.eval()  # 평가 모드
with torch.no_grad():
    for i in tqdm(range(0, len(input_ids), batch_size)):
        # 배치 추출
        input_ids_batch = input_ids[i:i+batch_size].to(device)
        attention_mask_batch = attention_mask[i:i+batch_size].to(device)
        true_labels_batch = true_labels[i:i+batch_size].to(device)

        # 모델 예측
        outputs = model_kobart_contents(input_ids=input_ids_batch, attention_mask=attention_mask_batch)

        # 각 클래스에 대한 확률 계산 (Softmax)
        probs = torch.softmax(outputs.logits, dim=-1)

        # 클래스 레이블 예측 (가장 높은 확률의 클래스를 선택)
        class_preds = torch.argmax(probs, dim=-1)

        # 양성 클래스 확률 (index 1)
        positive_class_probs = probs[:, 1]

        # 결과 저장
        all_class_preds.extend(class_preds.cpu().numpy())
        all_prob_preds.extend(positive_class_probs.cpu().numpy())
        all_true_labels.extend(true_labels_batch.cpu().numpy())

accuracy = calculate_accuracy(all_class_preds, all_true_labels)
print(f"KoBART Model Accuracy: {accuracy:.4f}")

auroc = calculate_auroc(all_prob_preds, all_true_labels)
print(f"KoBART Model AUROC: {auroc:.4f}")

precision = calculate_precision(all_class_preds, all_true_labels)
print(f"KoBART Model Precision: {precision:.4f}")

recall = calculate_recall(all_class_preds, all_true_labels)
print(f"KoBART Model Recall: {recall:.4f}")

f1 = calculate_f1(all_class_preds, all_true_labels)
print(f"KoBART Model F1 Score: {f1:.4f}")

100%|██████████| 997/997 [02:01<00:00,  8.23it/s]


KoBART Model Accuracy: 0.9995
KoBART Model AUROC: 0.9999
KoBART Model Precision: 0.9992
KoBART Model Recall: 0.9997
KoBART Model F1 Score: 0.9995


#### title 평가

In [117]:
# electra_contents tokenizer
from datasets import Dataset

tokenizer_KoELECTRA = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# 'doc_content' 컬럼을 토큰화하는 함수 정의
def tokenize_function(examples):
    examples['doc_title'] = [text + ' </s>' for text in examples['doc_title']]
    return tokenizer(examples['doc_title'], padding='max_length', max_length=256, truncation=True)

# DataFrame을 Hugging Face Dataset으로 변환
dataset_title = Dataset.from_pandas(test_title)

# 데이터셋에 토크나이저 적용
tokenized_test_title_electra = dataset_title.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_test_title_electra = tokenized_test_title_electra.rename_column("label", "labels")
tokenized_test_title_electra.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

In [118]:
# bart content tokenizer
from transformers import PreTrainedTokenizerFast
from datasets import Dataset

# KoBART 토크나이저 불러오기
tokenizer_KoBART = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

def tokenize_function(examples):
    examples['doc_title'] = [text + ' </s>' for text in examples['doc_title']]
    return tokenizer(examples['doc_title'], padding='max_length', max_length=128, truncation=True)

    return tokens

# DataFrame을 Hugging Face Dataset으로 변환
dataset_title = Dataset.from_pandas(test_title)

# 데이터셋에 토크나이저 적용
tokenized_test_title_kobart = dataset_title.map(tokenize_function, batched=True)

# 모델 학습에 필요한 컬럼 선택
tokenized_test_title_kobart = tokenized_test_title_kobart.rename_column("label", "labels")

# 모델에 필요한 형식으로 설정 (input_ids, attention_mask, labels)
tokenized_test_title_kobart.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

In [119]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def move_to_device(example):
    return {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in example.items()}

# 각 텐서를 GPU로 이동
tokenized_test_title_kobart = tokenized_test_title_kobart.map(move_to_device)
tokenized_test_title_electra = tokenized_test_title_electra.map(move_to_device)

# 모델을 GPU로 이동
model_kobart_title = model_kobart_contents.to(device)
model_koelectra_title = model_koelectra_context.to(device)

Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

Map:   0%|          | 0/15943 [00:00<?, ? examples/s]

In [120]:
# 예측 수행 및 라벨 추출 (확률 및 클래스 레이블 기반)
def get_prob_and_label_predictions(tokenized_kobart, tokenized_electra, model_kobart, model_electra):
    prob_preds = []
    class_preds = []
    true_labels = []

    # tqdm을 사용해 데이터셋 반복 시 진행 상황 표시
    for i in tqdm(range(len(tokenized_kobart)), desc="Processing Predictions"):
        # KoBART 데이터 추출
        kobart_input = {
            'input_ids': tokenized_kobart[i]['input_ids'].unsqueeze(0).to(device),
            'attention_mask': tokenized_kobart[i]['attention_mask'].unsqueeze(0).to(device)
        }

        # KoELECTRA 데이터 추출
        electra_input = {
            'input_ids': tokenized_electra[i]['input_ids'].unsqueeze(0).to(device),
            'attention_mask': tokenized_electra[i]['attention_mask'].unsqueeze(0).to(device)
        }

        # 모델 확률 및 클래스 레이블 예측
        prob_prediction, class_prediction = ensemble_predict_prob_and_label(kobart_input, electra_input, model_kobart, model_electra)

        # 실제 라벨 저장
        true_labels.append(tokenized_kobart[i]['labels'].item())

        # 확률 예측 및 클래스 레이블 저장
        prob_preds.append(prob_prediction.item())
        class_preds.append(class_prediction.item())

    return prob_preds, class_preds, true_labels

In [121]:
# 예측 수행
prob_predictions, class_predictions, true_labels = get_prob_and_label_predictions(
    tokenized_test_title_kobart, tokenized_test_title_electra, model_kobart_title, model_koelectra_title
)

accuracy = calculate_accuracy(class_predictions, true_labels)
print(f"Ensemble Model Accuracy: {accuracy:.4f}")

auroc = calculate_auroc(prob_predictions, true_labels)
print(f"Ensemble Model AUROC: {auroc:.4f}")

precision = calculate_precision(class_predictions, true_labels)
print(f"Ensemble Model Precision: {precision:.4f}")

recall = calculate_recall(class_predictions, true_labels)
print(f"Ensemble Model Recall: {recall:.4f}")

f1 = calculate_f1(class_predictions, true_labels)
print(f"Ensemble Modell F1 Score: {f1:.4f}")

Processing Predictions: 100%|██████████| 15943/15943 [07:52<00:00, 33.71it/s]

Ensemble Model Accuracy: 0.5773
Ensemble Model AUROC: 0.6173
Ensemble Model Precision: 0.6033
Ensemble Model Recall: 0.4437
Ensemble Modell F1 Score: 0.5113


In [114]:
input_ids = tokenized_test_title_electra['input_ids'].to(device)
attention_mask = tokenized_test_title_electra['attention_mask'].to(device)
true_labels = tokenized_test_title_electra['labels'].to(device)

batch_size = 16

# 예측 결과 저장용 리스트
all_class_preds = []
all_prob_preds = []
all_true_labels = []

# 데이터 배치 처리
model_koelectra_title.eval()  # 평가 모드
with torch.no_grad():
    for i in tqdm(range(0, len(input_ids), batch_size)):
        # 배치 추출
        input_ids_batch = input_ids[i:i+batch_size].to(device)
        attention_mask_batch = attention_mask[i:i+batch_size].to(device)
        true_labels_batch = true_labels[i:i+batch_size].to(device)

        # 모델 예측
        outputs = model_koelectra_title(input_ids=input_ids_batch, attention_mask=attention_mask_batch)

        # 각 클래스에 대한 확률 계산 (Softmax)
        probs = torch.softmax(outputs.logits, dim=-1)

        # 클래스 레이블 예측 (가장 높은 확률의 클래스를 선택)
        class_preds = torch.argmax(probs, dim=-1)

        # 양성 클래스 확률 (index 1)
        positive_class_probs = probs[:, 1]

        # 결과 저장
        all_class_preds.extend(class_preds.cpu().numpy())
        all_prob_preds.extend(positive_class_probs.cpu().numpy())
        all_true_labels.extend(true_labels_batch.cpu().numpy())

accuracy = calculate_accuracy(all_class_preds, all_true_labels)
print(f"KoELECTRA Model Accuracy: {accuracy:.4f}")

auroc = calculate_auroc(all_prob_preds, all_true_labels)
print(f"KoELECTRA Model AUROC: {auroc:.4f}")

precision = calculate_precision(all_class_preds, all_true_labels)
print(f"KoELECTRA Model Precision: {precision:.4f}")

recall = calculate_recall(all_class_preds, all_true_labels)
print(f"KoELECTRA Model Recall: {recall:.4f}")

f1 = calculate_f1(all_class_preds, all_true_labels)
print(f"KoELECTRA Model F1 Score: {f1:.4f}")

100%|██████████| 997/997 [00:53<00:00, 18.59it/s]

KoELECTRA Model Accuracy: 0.4984
KoELECTRA Model AUROC: 0.5599
KoELECTRA Model Precision: 0.4984
KoELECTRA Model Recall: 1.0000
KoELECTRA Model F1 Score: 0.6652


In [122]:
input_ids = tokenized_test_title_kobart['input_ids'].to(device)
attention_mask = tokenized_test_title_kobart['attention_mask'].to(device)
true_labels = tokenized_test_title_kobart['labels'].to(device)

batch_size = 16

# 예측 결과 저장용 리스트
all_class_preds = []
all_prob_preds = []
all_true_labels = []

# 데이터 배치 처리
model_kobart_title.eval()  # 평가 모드
with torch.no_grad():
    for i in tqdm(range(0, len(input_ids), batch_size)):
        # 배치 추출
        input_ids_batch = input_ids[i:i+batch_size].to(device)
        attention_mask_batch = attention_mask[i:i+batch_size].to(device)
        true_labels_batch = true_labels[i:i+batch_size].to(device)

        # 모델 예측
        outputs = model_kobart_title(input_ids=input_ids_batch, attention_mask=attention_mask_batch)

        # 각 클래스에 대한 확률 계산 (Softmax)
        probs = torch.softmax(outputs.logits, dim=-1)

        # 클래스 레이블 예측 (가장 높은 확률의 클래스를 선택)
        class_preds = torch.argmax(probs, dim=-1)

        # 양성 클래스 확률 (index 1)
        positive_class_probs = probs[:, 1]

        # 결과 저장
        all_class_preds.extend(class_preds.cpu().numpy())
        all_prob_preds.extend(positive_class_probs.cpu().numpy())
        all_true_labels.extend(true_labels_batch.cpu().numpy())

accuracy = calculate_accuracy(all_class_preds, all_true_labels)
print(f"KoBART Model Accuracy: {accuracy:.4f}")

auroc = calculate_auroc(all_prob_preds, all_true_labels)
print(f"KoBART Model AUROC: {auroc:.4f}")

precision = calculate_precision(all_class_preds, all_true_labels)
print(f"KoBART Model Precision: {precision:.4f}")

recall = calculate_recall(all_class_preds, all_true_labels)
print(f"KoBART Model Recall: {recall:.4f}")

f1 = calculate_f1(all_class_preds, all_true_labels)
print(f"KoBART Model F1 Score: {f1:.4f}")

100%|██████████| 997/997 [00:32<00:00, 30.92it/s]

KoBART Model Accuracy: 0.5015
KoBART Model AUROC: 0.6290
KoBART Model Precision: 0.0000
KoBART Model Recall: 0.0000
KoBART Model F1 Score: 0.0000
